In [ ]:
cd /home/stlk/Desktop/DigEc_data

## unzipping folders

for f in *.zip; 
do mkdir "${f%}_folder" && unzip "$f" -d "${f%}_folder" && mv "${f%}_folder"/* "${f%.zip}.csv"; done

In [28]:
## creating list of absolute paths to csvs

ls -d -1 $PWD/*/*.csv >> /home/stlk/Desktop/list.txt
ls -d -1 $PWD/*.csv >> /home/stlk/Desktop/list_creators.txt

ERROR: Error in parse(text = x, srcfile = src): <text>:1:4: unexpected symbol
1: cd Desktop
       ^


In [1]:
## stacking multiple csvs 
### not working till csvs have different number of columns

csvstack /home/stlk/Desktop/DigEc_data/*/Kickstarter*.csv --filenames >> /home/stlk/Desktop/DigEc_data/un.csv

csvstack /home/stlk/Desktop/DigEc_scripts/mongo_exp/*.csv --filenames >> /home/stlk/Desktop/DigEc_scripts/mongo_exp/creators_united.csv

ERROR: Error in parse(text = x, srcfile = src): <text>:1:40: unexpected '*'
1: csvstack /home/stlk/Desktop/DigEc_data/*
                                           ^


In [2]:
## mySQL for creating and importing data


CREATE DATABASE kck;

USE KCK;


CREATE TABLE `kck`
(`group` VARCHAR(100),
`id` INT,
PRIMARY KEY (`id`),
`photo` JSON,
`name` MEDIUMTEXT,
`blurb` MEDIUMTEXT,
`goal` INT,
`pledged` FLOAT(15,15),
`state` VARCHAR(80),
`slug` VARCHAR(1000),
`disable_communication` BOOLEAN,
`country` VARCHAR(100),
`currency` VARCHAR(100),
`currency_symbol` VARCHAR(100),
`currency_trailing_code` BOOLEAN,
`deadline` INT,
`state_changed_at` INT,
`created_at` INT,
`launched_at` INT,
`staff_pick` BOOLEAN,
`is_starrable` BOOLEAN,
`backers_count` INT,
`static_usd_rate` FLOAT(15,15),
`usd_pledged` FLOAT(15,15),
`converted_pledged_amount` INT,
`current_currency` VARCHAR(100),
`usd_type` VARCHAR(100),
`creator` MEDIUMTEXT,
`location` MEDIUMTEXT,
`category` MEDIUMTEXT,
`profile` MEDIUMTEXT,
`spotlight` BOOLEAN,
`urls` MEDIUMTEXT,
`source_url` MEDIUMTEXT,
`friends` VARCHAR(10),
`is_starred` VARCHAR(10),
`is_backing` VARCHAR(10),
`permissions` VARCHAR(10)
);


LOAD DATA LOCAL INFILE "/home/stlk/Desktop/DigEc_data/un.csv"
INTO TABLE `kck`
COLUMNS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: CREATE DATABASE
           ^


In [ ]:
## monogdb syntax

sudo systemctl start mongodb && sudo systemctl status mongodb

    mongo 

use kck

db.createCollection('kck')
db.createCollection('KCK')

ls -d -1 $PWD/*/*.csv >> /home/stlk/Desktop/list.txt
ls -d -1 $PWD/*.csv >> /home/stlk/Desktop/list_KCK.txt



In [ ]:
## Adding path columns to csvs

a=1
IFS=$'\n'
for f in $(cat /home/stlk/Desktop/list.txt);
do Rscript --vanilla /home/stlk/Desktop/add_path.r "$f" "$a" &&
echo $((a+=1)); done


In [ ]:
## importing set of csv to mongo

a=0
IFS=$'\n'
for f in $(cat /home/stlk/Desktop/list.txt);
do mongoimport -d kck -c kck --type csv --file "$f" --headerline &&
echo $((a+=1)); done

In [ ]:
# mongoimport -d kck -c df_test --type csv --file /home/stlk/Desktop/d1.csv --headerline
# mongoimport -d kck -c df_test --type csv --file /home/stlk/Desktop/d2.csv --headerline

# mongoimport -d kck -c kck_test --type csv --file /home/stlk/Desktop/DigEc_data/Kickstarter_2017-10-15T10_20_38_271Z.zip_folder/Kickstarter001.csv --headerline
# mongoimport -d kck -c kck_test --type csv --file /home/stlk//Desktop/DigEc_data/Kickstarter_2016-01-28T09_15_08_781Z.zip_folder/Kickstarter001.csv --headerline

mongoimport -d kck -c kck --type csv --file /home/stlk/Desktop/DigEc_data/un.csv --headerline

In [ ]:
## exporting csv out of mongo

# mongoexport --db kck --collection kck_test --jsonArray > /home/stlk/Desktop/test1.json

# print number of lines
awk '{n+=1} END {print n}' Desktop/DigEc_data/un.csv

In [27]:
## checking ids

# export column
csvcut -c id Desktop/DigEc_data/un.csv >> Desktop/DigEc_data/ids_un.csv

# export column mongo
mongoexport --host localhost --db kck --collection kck --csv --out /home/stlk/Desktop/DigEc_data_samples/text_mongo.json --fields country

# export y from mongo
## non-nested fields
mongoexport --host localhost --db kck --collection kck --csv --out /home/stlk/Desktop/DigEc_data_samples/y_mongo.csv --fields id,blurb,goal,pledged,state,state_changed_at

# export for missing state_changed_at
mongoexport --host localhost --db kck --collection KCK --csv --out /home/stlk/Desktop/DigEc_scripts/mongo_exp/creators_with_time.csv --fields id,creator_id,state_changed_at

ERROR: Error in parse(text = x, srcfile = src): <text>:3:5: unexpected symbol
2: 
3: use kck
       ^


In [1]:
## checking stacked csv exported from mySQL and before being imported

ids$id %>% unique() %>% length()
ids_mongo$id %>% unique() %>% length()

ids_mongo %>% nrow()
ids %>% nrow()

# setdiff(ids, ids_mongo) %>% head()

ERROR: Error in ids$id %>% unique() %>% length(): could not find function "%>%"


In [3]:
## extract all field names

mr = db.runCommand({
  "mapreduce" : "kck",
  "map" : function() {
    for (var key in this) { emit(key, null); }
  },
  "reduce" : function(key, stuff) { return null; },
  "out": "kck" + "_keys"
})

db[mr.result].distinct("_id")

ERROR: Error in parse(text = x, srcfile = src): <text>:4:22: unexpected ','
3: mr = db.runCommand({
4:   "mapreduce" : "kck",
                        ^


In [ ]:
## count number of values

db.kck.aggregate(
   {$group : { _id : '$country', count : {$sum : 1}}}
)


In [ ]:
mongoexport --host localhost --db kck --collection KCK --csv --out /home/stlk/Desktop/DigEc_Kickstarter/SocMedia_Conf/full.json --fields id,blurb,state,state_changed_at,backers_count,goal,static_usd_rate,usd_pledged,creator_id,category_slug

id, blurb, state, state_changed_at, backers_count, goal, static_usd_rate, usd_pledged, creator_id, category_slug